<a href="https://colab.research.google.com/github/diablo-1997/GoogleColab/blob/main/AngelOne_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Origianl API call from AngelOne.
```python
# Define the URL and payload
url = "https://www.angelone.in/wp-json/facetwp/v1/refresh"
payload = {
    "action": "facetwp_refresh",
    "data": {
        "facets": {
            "stock_search_by_alphabet": [],
            "stock_universe": [],
            "stock_sector": [],
            "stock_market_cap_by_value": [],
            "stock_market_cap_by_size": [],
            "stock_result_count_pager": [],
            "stock_archive_per_page_pager": [],
            "stock_pagination": []
        },
        "frozen_facets": {},
        "http_params": {
            "get": {},
            "uri": "stocks",
            "url_vars": []
        },
        "template": "stock_archive_data",
        "extras": {
            "per_page": 100,
            "selections": True,
            "sort": "default"
        },
        "soft_refresh": 0,
        "is_bfcache": 1,
        "first_load": 0,
        "paged": 1
    }
}

# Send the POST request
response = requests.post(url, json=payload)

# Check the response status code
if response.status_code == 200:
  # Request successful!
  print("Request successful! Response:", response.text)
else:
  # There was an error
  print("Error:", response.status_code, response.text)
```

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook


In [ ]:
BASE_URL = "https://www.angelone.in/"
MAIN_ROOT = "wp-json/facetwp/v1/refresh"


In [ ]:
def api(payload, base_url=BASE_URL, root=MAIN_ROOT):
  url = base_url + root

  try:
    response = requests.post(url, json=payload)
    response.raise_for_status()  # Raise exception for unsuccessful requests (improves error handling)
  except requests.exceptions.RequestException as e:
    return {"Error": f"Error sending request: {e}"}

  return response.json()


In [ ]:
def fetch_stock_sectors():
  """
  Fetches stock sectors from Angel Broking website.

  Returns:
      dict: Dictionary containing stock sector names as keys and data-values as values.
          If there's an error, returns a dictionary with error details.
  """
  payload = {
      "action": "facetwp_refresh",
      "data": {
          "facets": {"stock_sector": []},
          "template": "stock_archive_data",
      }
  }

  data = api(payload)
  sector_html = data.get("facets", {}).get("stock_sector", "")

  soup = BeautifulSoup(sector_html, 'html.parser')
  sectors = dict()
  for sector in soup.find_all('div', class_='facetwp-radio'):
    data_value = sector.get('data-value')
    if data_value:
      sector_text = sector.find("span", class_="facetwp-display-value").text.strip()
      sectors[sector_text] = data_value

  return sectors


# Example usage
# fetch_stock_sectors()

In [ ]:
def fetch_company_names(stock_sector):
    payload = {
        "action": "facetwp_refresh",
        "data": {
            "facets": {"stock_sector": [stock_sector],},
            "http_params": {
                "get": {"_stock_sector": stock_sector},
                "uri": "stocks",
                "url_vars": []
            },
            "template": "stock_archive_data",
            "extras": {
                "per_page": "1000",
                "selections": True,
                "sort": "default"
            },
            "paged": 1,
        }
    }

    data = {}

    template_html = api(payload).get("template", "")
    soup = BeautifulSoup(template_html, 'html.parser')

    tbody = soup.find('tbody')
    tr = tbody.find_all('tr')

    for r in tr:
      data[r.find("p", attrs={"class": "company-short-name"}).text.strip()] = r.find("p", attrs={"class": "company-name"}).text.strip()

    return data


# Example usage
# fetch_company_names("logistics")


In [ ]:
def write_sector_data_to_excel(workbook, sectors):
  """
  Writes sector and company data to an Excel workbook.

  Args:
      workbook: An openpyxl Workbook object.
      sectors: A dictionary where keys are sector names and values are lists of companies in that sector.
  """
  worksheet = workbook.active
  worksheet.append(["Sector", "Company"])  # Add header row

  for sector, companies in sectors.items():
    for company in companies:
      worksheet.append([sector, company])


def main():
  sectors = fetch_stock_sectors()
  company_data_by_sector = {str(sector).replace("-", " ").title(): fetch_company_names(sector) for sector in sectors.values()}

  # Create a new workbook and write data
  workbook = Workbook()
  write_sector_data_to_excel(workbook, company_data_by_sector)
  workbook.save("frameData.xlsx")


In [ ]:
if __name__ == "__main__":
    main()